# YOLOv5

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
root_dir = '/content/drive/MyDrive/krfood/yolov5'
os.chdir(root_dir)

In [3]:
import zipfile
data_dir = os.path.join(root_dir, 'data')
food_zip = zipfile.ZipFile(os.path.join(data_dir, 'krfood.zip'))

unzip_dir = os.path.join(data_dir, 'krfood')
if os.path.exists(unzip_dir):
  shutil.rmtree(unzip_dir)

food_zip.extractall(data_dir) # 압축 해제 9분 소요

In [ ]:
%pip install -r requirements.txt

## Train

In [6]:
%python train.py --data "data/krfood.yaml" --cfg "data/yolov5s.yaml" --epochs 3 --batch-size 64

train: weights=yolov5s.pt, cfg=, data=data/krfood.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2022-5-23 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.01

In [ ]:
%zip -r train_result.zip ./runs/train/exp

## Validate

In [18]:
%python val.py --data "data/krfood.yaml" --weights "./runs/train/exp/weights/best.pt"

val: data=data/krfood.yaml, weights=['./runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 2022-5-23 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7414675 parameters, 0 gradients, 17.1 GFLOPs
val: Scanning '/content/drive/MyDrive/Final/yolov5/krfood/valid/labels/0.cache' images and labels... 5153 found, 0 missing, 0 empty, 10 corrupt: 100% 5163/5163 [00:00<?, ?it/s]
val: WARNING: /content/drive/MyDrive/Final/yolov5/krfood/valid/images/100/Img_023_0465.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/Final/yolov5/krfood/valid/images/103/Img_055_0271.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/Final/yolov5/krfood/v

In [ ]:
%zip -r val_result.zip ./runs/val

## Test

In [39]:
%python detect.py --source "./krfood/test/images/100" --weights "./runs/train/exp/weights/best.pt"

detect: weights=['./runs/train/exp/weights/best.pt'], source=./krfood/test/images/4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2022-5-23 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7414675 parameters, 0 gradients, 17.1 GFLOPs
image 1/30 /content/drive/MyDrive/Final/yolov5/krfood/test/images/4/Img_134_0027.jpg: 640x480 Done. (0.017s)
image 2/30 /content/drive/MyDrive/Final/yolov5/krfood/test/images/4/Img_134_0031.jpg: 640x640 Done. (0.014s)
image 3/30 /content/drive/MyDrive/Final/yolov5/krfood/test/images/4/Img_134_0033.jpg: 640x480 Done. (0.012s)
image 4/30 /content/drive/MyDrive/Fi

In [ ]:
%zip -r test_result.zip ./runs/detect/exp5

## Result

In [ ]:
import torch
from matplotlib import pyplot as plt
import cv2
from PIL import Image

#Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./runs/train/exp/weights/best.pt')  # local repo
# Images
img = cv2.imread('../data/test/images/100/Img_023_0064.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Inference
results = model(img, size=328)  # includes NMS

#results = results.xyxy[0]  # img1 predictions (tensor)
boxes = results.pandas().xyxy[0]  # img1 predictions (pandas)

# Results
results.print()  
results.show()  # or .show()

In [ ]:
image_path = './runs/detect/exp/Img_134_0224.jpg'
image = cv2.imread(image_path)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()